In [5]:
!pip install pandas

You should consider upgrading via the 'D:\Data Engineer\Portafolio Lorenzo Guerrero\Proyecto ETL LG\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import pandas as pd



In [7]:
df_sales = pd.read_csv('data/sales_data_sample.csv',sep =',',encoding = 'latin1')


In [8]:
#get the original columns
columns = df_sales.columns

In [9]:
#Create a list with the new columns , in lowercase
columns_df = columns.map(lambda x: x.lower())

In [10]:
#Change the columns
df_sales.columns = columns_df

In [11]:
#Modify columns same with the right valor
df_sales['sales'] = df_sales['quantityordered']*df_sales['priceeach']


In [12]:
#fill values na in column territory
def territory_set(value):
    if pd.isna(value):
        return 'AMERICA'
    else:
        return value
df_sales['territory'] = df_sales['territory'].apply(territory_set)

In [13]:
#Get the values uniques of territory and create a primary key
territory = df_sales.get(['territory'])
territories = territory.drop_duplicates().reset_index(drop=True)
territories['territory_id'] = [i + 1 for i in range(len(territories))]
territories = territories.get(['territory_id','territory'])

In [14]:
#Get the values uniques of countries and create primary key  and merge to get foreing key
country = df_sales.get(['country','territory'])
countries = country.drop_duplicates(subset = ['country']).reset_index(drop=True)
countries['country_id'] = pd.factorize(countries['country'])[0] + 1
countries = countries.get(['country_id','country','territory'])
countries = countries.merge(territories, on='territory', how='left')
countries = countries.get(['country_id','country','territory_id'])

In [15]:
#Get the values uniques of citys and create primary key  and merge to get foreing key
cities = df_sales.get(['city','state','country'])
cities['city_id'] = pd.factorize(cities['city'])[0] + 1
citys = cities.merge(countries.get(['country_id','country']), on='country', how='left') #first do the merge and then delete duplicates
citys = citys.drop_duplicates(subset=['city','country']).reset_index(drop=True)
citys = citys.get(['city_id','city','state','country_id'])

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_3192\653603705.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities['city_id'] = pd.factorize(cities['city'])[0] + 1


In [16]:
#Get the values uniques of customers and create primary key  and merge to get foreing key
customers = df_sales.get(['customername','contactlastname','contactfirstname','phone','addressline1','addressline2','postalcode','city'])
customers['customer_id'] = pd.factorize(customers['customername'])[0] + 1
customers = customers.merge(citys.get(['city_id','city']), on='city', how='left')
customers = customers.drop_duplicates(subset=['customername','phone']).reset_index(drop=True)
customers = customers.get(['customer_id','customername','contactlastname','contactfirstname','phone','addressline1','addressline2','postalcode','city_id'])


C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_3192\3534362300.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['customer_id'] = pd.factorize(customers['customername'])[0] + 1


In [17]:
#Get the values uniques of status and create primary key
status = df_sales.get(['status'])
status = status.drop_duplicates().reset_index(drop=True)
status['status_id'] = [i + 1 for i in range(len(status))]
status = status.get(['status_id','status'])

In [18]:
#Get the values uniques of orders and create primary key and merge to get foreing key
orders = df_sales.get(['ordernumber','orderdate','qtr_id','month_id','year_id','status','customername']).sort_values(by='ordernumber')
orders['order_id'] = pd.factorize(orders['ordernumber'])[0] + 1
orders = orders.merge(status.get(['status_id','status']), on='status', how='left')
orders = orders.merge(customers.get(['customer_id','customername']), on='customername', how='left')
orders = orders.drop_duplicates(subset=['ordernumber','orderdate']).reset_index(drop=True)
orders['orderdate'] = pd.to_datetime(orders['orderdate']) #Change format to YYYY-MM-DD
orders = orders.get(['order_id','ordernumber','orderdate','qtr_id','month_id','year_id','status_id','customer_id'])

In [19]:
#Get the values uniques of ProductLines
product_lines = df_sales.get(['productline'])
product_lines = product_lines.drop_duplicates().reset_index(drop=True)
product_lines['productline_id'] = [i + 1 for i in range(len(product_lines))]
product_lines = product_lines.get(['productline_id','productline'])


In [20]:
#Get the values uniques of dealsizes
dealsizes = df_sales.get(['dealsize'])
dealsizes  = dealsizes .drop_duplicates().reset_index(drop=True)
dealsizes ['dealsize_id'] = [i + 1 for i in range(len(dealsizes))]
dealsizes  = dealsizes .get(['dealsize_id','dealsize'])

In [21]:
#Get the values uniques of products and create primary key  and merge to get foreing key
products = df_sales.get(['productcode','msrp','productline'])
products['product_id'] = pd.factorize(products['productcode'])[0] + 1
products = products.merge(product_lines.get(['productline_id','productline']), on='productline', how='left')
products = products.drop_duplicates(subset=['productcode','msrp']).reset_index(drop=True)
products = products.get(['product_id','productcode','msrp','productline_id',])

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_3192\3532212272.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['product_id'] = pd.factorize(products['productcode'])[0] + 1


In [22]:
#Create table order_products
order_products = df_sales.get(['ordernumber','productcode','quantityordered','priceeach','orderlinenumber','sales','dealsize','customername']).sort_values(by='ordernumber')
order_products = order_products.merge(products.get(['product_id','productcode']), on='productcode', how='left')
order_products = order_products.merge(orders.get(['order_id','ordernumber']), on='ordernumber', how='left')
order_products = order_products.merge(dealsizes.get(['dealsize_id','dealsize']), on='dealsize', how='left')
order_products = order_products.get(['order_id','product_id','quantityordered','priceeach','orderlinenumber','sales','dealsize_id'])

In [23]:
data = {'territories':territories,'countries':countries,'citys':citys,'customers':customers,'status':status,'orders':orders,'product_lines':product_lines,
        'dealsizes':dealsizes,'products':products,'order_products':order_products}

In [25]:
for df_name, df in data.items():
    df.to_csv(f'output_Data/{df_name}.csv',index=False)
    print(f'Dataframe {df_name} successfuly convert to csv')


Dataframe territories successfuly convert to csv
Dataframe countries successfuly convert to csv
Dataframe citys successfuly convert to csv
Dataframe customers successfuly convert to csv
Dataframe status successfuly convert to csv
Dataframe orders successfuly convert to csv
Dataframe product_lines successfuly convert to csv
Dataframe dealsizes successfuly convert to csv
Dataframe products successfuly convert to csv
Dataframe order_products successfuly convert to csv
